# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

In [14]:
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
city_data = "../WeatherPy/Data/clean_city_data.csv"
city_data = pd.read_csv(city_data)
city_data

,Unnamed: 0,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City
0,0,-33.5906,26.8910,12.17,75,100,10.42,ZA,1626831878,port alfred
1,1,54.0359,19.0266,14.93,94,48,3.29,PL,1626831879,malbork
2,2,-34.5322,20.0403,6.62,81,53,2.57,ZA,1626831879,bredasdorp
3,3,-0.7393,-90.3518,22.53,94,100,3.13,EC,1626831879,puerto ayora
4,4,-37.8333,140.7667,9.95,81,90,4.63,AU,1626831880,mount gambier
...,...,...,...,...,...,...,...,...,...,...
668,668,36.2970,59.6062,23.08,40,0,3.09,IR,1626832112,mashhad
669,669,13.6017,26.6876,28.33,52,100,5.97,SD,1626832113,umm kaddadah
670,670,41.6892,122.1128,28.08,75,74,5.31,CN,1626832113,heishan
671,671,41.9301,2.2549,24.06,66,11,0.89,ES,1626832114,vic


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Store latitude and longitude in locations
locations = city_data[["Lat", "Long"]]

# Fill NaN values and convert to float
humidity = city_data["Humidity"].astype(float)

In [18]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=True, max_intensity=100,
                                 point_radius=10)


# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
#Ideal Weather
clean_weather_data = city_data.loc[city_data["Wind Speed"] < 10]
clean_weather_data = clean_weather_data.loc[clean_weather_data["Humidity"] < 30]
clean_weather_data = clean_weather_data.loc[clean_weather_data["Max Temp"] > 20]
clean_weather_data = clean_weather_data.loc[clean_weather_data["Max Temp"] < 30]
clean_weather_data

,Unnamed: 0,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City
39,39,-20.7333,139.5000,21.87,24,20,7.20,AU,1626831892,mount isa
74,74,28.7519,54.5444,29.48,22,17,2.85,IR,1626831903,darab
102,102,68.2191,-135.0107,29.46,25,8,3.72,CA,1626831914,aklavik
196,196,47.8667,88.1167,27.70,22,1,1.03,CN,1626831947,altay
265,265,-20.1000,146.2667,21.16,24,31,6.50,AU,1626831970,charters towers
299,299,-12.0667,134.2667,28.95,28,68,5.28,AU,1626831982,maningrida
322,322,-15.7667,128.7333,27.99,16,100,4.12,AU,1626831989,kununurra
466,466,-14.4667,132.2667,26.18,25,34,5.14,AU,1626832039,katherine
591,591,33.9831,51.4364,28.98,18,0,1.52,IR,1626832083,kashan
592,592,-20.0167,148.2333,23.12,27,39,2.23,AU,1626832083,bowen


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
#Condensing the DF
hotel_df = clean_weather_data[["City", "Country","Lat", "Long"]].copy()
hotel_df

,City,Country,Lat,Long
39,mount isa,AU,-20.7333,139.5000
74,darab,IR,28.7519,54.5444
102,aklavik,CA,68.2191,-135.0107
196,altay,CN,47.8667,88.1167
265,charters towers,AU,-20.1000,146.2667
299,maningrida,AU,-12.0667,134.2667
322,kununurra,AU,-15.7667,128.7333
466,katherine,AU,-14.4667,132.2667
591,kashan,IR,33.9831,51.4364
592,bowen,AU,-20.0167,148.2333


In [21]:
#Adding Hotel Column
hotel_df["Hotel Name"] = ""

hotel = []
#BaseURL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#Setting the Params
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "Lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    lat = row["Lat"]
    lon = row["Long"]
    coord = f"{lat},{lon}"
    
    params["location"] = coord

    #Make API request
    print(f"Retrieving Results for Index {index}: {coord}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]["name"]
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [22]:
#Export CSV
# hotel_df.to_csv("Data/hotel_data.csv")
hotel_df

,City,Country,Lat,Long,Hotel Name
39,mount isa,AU,-20.7333,139.5000,Isa Hotel
74,darab,IR,28.7519,54.5444,Naghshe shapoor Hotel
102,aklavik,CA,68.2191,-135.0107,Sundog Inn
196,altay,CN,47.8667,88.1167,阿勒泰市金都酒店
265,charters towers,AU,-20.1000,146.2667,Charters Towers Tourist Park
299,maningrida,AU,-12.0667,134.2667,Dhukurrdji Lodge
322,kununurra,AU,-15.7667,128.7333,Freshwater East Kimberley Apartments | Kununur...
466,katherine,AU,-14.4667,132.2667,Contour Hotel Katherine
591,kashan,IR,33.9831,51.4364,Givak hotel
592,bowen,AU,-20.0167,148.2333,Queens Beach Hotel


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [24]:
# Create a marker_layer for hotels
markers = gmaps.marker_layer(locations, hotel_info)
fig.add_layer(markers)

fig



# fig_two = gmaps.figure()
# markers = gmaps.marker_layer(locations,
#     hotel_info)
# fig_two.add_layer(markers)
# fig_two

Figure(layout=FigureLayout(height='420px'))